In [2]:
import os 
import json
import pandas as pd
import numpy as np

os.chdir('/home/jonah/Desktop/DataVizDashboard/data/')

In [3]:
with open('revenue_by_rating_and_genre.json') as file:
    dict = json.load(file)

data = pd.DataFrame(dict)

In [4]:
df = pd.read_csv('credits.csv')
df1 = df['cast'].str.split(', ', expand=True)
df1 = pd.DataFrame(df1)
df1['id'] = df['id']
df2 = df1[3]
df2 = pd.DataFrame(df2)
df2['id'] = df['id']
df2 = df2.rename(columns={3: 'gender'})
df2['gender'] =df2['gender'].str.replace("'gender':", '', regex=True)
df2.dropna(inplace=True)
df2['gender'] = df2['gender'].str.replace('2', 'Male')
df2['gender'] =df2['gender'].str.replace('1', 'Female')
df2.drop_duplicates(subset='id', keep='first', inplace=True)
df2

,gender,id
0,Male,862
1,Male,8844
2,Male,15602
3,Female,31357
4,Male,11862
...,...,...
45470,Male,30840
45471,Female,439050
45472,Female,111109
45473,Female,67758


In [5]:
ratings_df = pd.read_csv('ratings_small.csv')
ratings_df = ratings_df.drop(columns=['timestamp'])
ratings_df = ratings_df.drop(columns=['userId'])
ratings_df = ratings_df.rename(columns={'movieId': 'id'})
ratings_df.drop_duplicates(subset='id', keep='first', inplace=True)
ratings_df = ratings_df.reset_index(drop=True)
ratings_df

,id,rating
0,31,2.5
1,1029,3.0
2,1061,3.0
3,1129,2.0
4,1172,4.0
...,...,...
9061,64997,2.5
9062,72380,3.5
9063,129,3.0
9064,4736,1.0


In [6]:
df3 = pd.merge(df2, ratings_df, on='id')

In [7]:
genre_df = pd.read_csv('movies_metadata.csv')
df4 = genre_df[['id', 'genres', 'revenue']]
df5 = df4['genres'].str.split(',', expand=True)
df4['genres'] = df5[0]
df4.dropna(inplace=True)
df4.drop_duplicates(subset='id', keep='first', inplace=True)
df4 = df4.reset_index(drop=True)

/tmp/ipykernel_5147/3499409350.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['genres'] = df5[0]
/tmp/ipykernel_5147/3499409350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.dropna(inplace=True)
/tmp/ipykernel_5147/3499409350.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop_duplicates(subset='id', keep='first', inplace=True)


In [8]:
df5 = pd.merge(df3, df4, on='id')
df5['gender'] = df5['gender'].str.replace(' 0', ' Male')
# df5['gender'] = df5['gender'].str.replace("'credit_id': '5Male'", ' Male')
df5['gender'] = np.where((df5['gender'] != ' Female') & (df5['gender'] != ' Male'), ' Male', df5['gender'])
df5 = df5.drop(columns=['id'])

In [12]:
pivot_df = pd.pivot_table(df5, values='revenue', index=['gender', 'rating'], columns='genres', aggfunc='mean')
pivot_df = pivot_df.reset_index()

pivot_df.columns.name = None

print(pivot_df)

     gender  rating        Action     Adventure     Animation        Comedy  \
0    Female     0.5  5.248158e+06           NaN           NaN  0.000000e+00   
1    Female     1.0  1.001732e+07  5.188101e+07           NaN  8.307200e+07   
2    Female     1.5  0.000000e+00           NaN           NaN  1.856758e+08   
3    Female     2.0  1.605067e+08  1.469809e+08           NaN  6.312752e+07   
4    Female     2.5  3.300987e+06  0.000000e+00           NaN  1.364136e+07   
5    Female     3.0  9.430973e+07  5.500000e+08  4.118853e+06  4.053880e+07   
6    Female     3.5  1.809490e+08  9.095774e+07  0.000000e+00  1.637194e+07   
7    Female     4.0  1.634323e+07  1.446405e+07           NaN  4.202135e+07   
8    Female     4.5  1.386364e+08  1.415093e+08           NaN  1.175344e+08   
9    Female     5.0  1.594025e+08  1.861174e+08           NaN  6.189717e+07   
10     Male     0.5  4.042264e+07  1.377501e+08  0.000000e+00  9.638346e+06   
11     Male     1.0  7.667958e+07  1.726108e+08  2.7

In [13]:
df6 = pd.DataFrame(pivot_df)
df6 = df6.fillna(0)


In [14]:
df6.to_csv('q2_data.csv', index=False)

In [20]:
df_female = df6[df6['gender'] == ' Female'].drop(columns='gender')
df_male = df6[df6['gender'] == ' Male'].drop(columns='gender')

In [22]:
df_female.to_csv('q2_female.csv', index=False)
df_male.to_csv('q2_male.csv', index=False)
